In [11]:
from Crypto.Cipher import AES
from Crypto.Protocol.KDF import scrypt
from Crypto.Util.number import long_to_bytes
import random
from random import randint
import pickle
s=10 # length of secret key
n=24 ## lenght of a message
N=2**n ## 2^n
padding = '00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

In [7]:
def gen_unique_keys2(s,N):
    unique_keys=[]
    x=0
    for _ in range(0, s):
        x= x *10 + 9
    unique_keys = random.sample(range(10**(s-1), x), N)
    return unique_keys

In [13]:

secret_keys = []
secret_messages = []
secret_keys = gen_unique_keys2(s, N) 
secret_messages = gen_unique_keys2((n-s), N)

generated
generated


In [76]:
def generate_puzzles():

    puzzles = []
    results = []
    print(N)
    for i in range(0, N):
        aes_key = randint(2**10, (2**11) - 1)
        #key = scrypt(long_to_bytes(aes_key), b"code", 32, N = 2 ** 2, r = 8, p = 1)
        if i % 100000 == 0 :
            print(i)
        x_key = str(bin(aes_key)[2:]) + padding
        b_key = long_to_bytes(int(x_key,2))
        cipher = AES.new(b_key, AES.MODE_GCM)
        sm = str(secret_messages[i]).encode("utf8") + str(secret_keys[i]).encode("utf8") 
        ciphertext, tag = cipher.encrypt_and_digest(sm)

        enc = cipher.nonce + ciphertext + tag
        results.append([secret_messages[i], secret_keys[i]])
        puzzles.append(enc)

    with open('./wyniki_24.pkl', 'wb') as f:
        pickle.dump(results, f)
    with open('./puzzle_24.pkl', 'wb') as f:
        pickle.dump(puzzles, f)

generate_puzzles()


16777216
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000

In [16]:
import gc
import _pickle as pickle

def load_cpickle_gc(x_file):
    output = open(x_file, 'rb')
    # disable garbage collector
    gc.disable()
    mydict = pickle.load(output)
    # enable garbage collector again
    gc.enable()
    output.close()
    return mydict
    
wyniki_file = load_cpickle_gc('./wyniki_24.pkl')
puzzle_file = load_cpickle_gc('./puzzle_24.pkl')

In [17]:

def decrypt_random():
    i = randint(0, len(puzzle_file)-1)
    print(i)
    key = puzzle_file[i]
    data = puzzle_file[i]
    counter = 0
    ## bruteforce szukam klucza do odszyfrowania
    for j in range(2**10, 2**11):
        aes_key = j
        #key = scrypt(long_to_bytes(aes_key), b"code", 32, N = 2 ** 10, r = 8, p = 1)
        x_key = str(bin(aes_key)[2:]) + padding
        b_key = long_to_bytes(int(x_key,2))
        cipher = AES.new(b_key, AES.MODE_GCM, data[:16]) # nonce
        try:
            dec = cipher.decrypt_and_verify(data[16:-16], data[-16:]) # ciphertext, tag
            calc_key = dec[-s:]
            calc_msg = dec[0:-s]
            return calc_msg, calc_key
        except ValueError:
            counter += 1
    print("nie znalezionio klucza")
    return 0,0


msg, key = decrypt_random()


3712644


In [13]:
def verify_key(key1, key2):
    return key1 == key2

In [18]:
def find_key_for_msg(msg):
    for x in range(0, len(wyniki_file)):
        if int(wyniki_file[x][0]) == msg:
            return wyniki_file[x][1]

In [19]:
found_key = find_key_for_msg(int(msg))

verify_key(found_key, int(key))

True